<a href="https://colab.research.google.com/github/ryyhan/fineTunedLLM/blob/main/GPT_2_Fine_Tuned(ConversationsClusters).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process for the First 3-Clustered Dataset

In [10]:
!pip install transformers datasets evaluate torch

In [11]:
import pandas as pd

# Load your dataframe
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsClusters)/clustered_df3.csv')  # Replace with your data file
df.drop(columns=['Unnamed: 0'], inplace=True)
df.columns=['conversation','cluster']

In [18]:
# Split the data into train and test sets
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
from transformers import GPT2Tokenizer

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [19]:
# Tokenize the conversations
max_length = 128  # Adjust based on your data
train_encodings = tokenizer(train_df['conversation'].tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(test_df['conversation'].tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

In [20]:
#creating a custom dataset

import torch
from torch.utils.data import Dataset

class ConversationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the datasets
train_dataset = ConversationDataset(train_encodings, train_df['cluster'].tolist())
test_dataset = ConversationDataset(test_encodings, test_df['cluster'].tolist())

In [15]:
from transformers import GPT2ForSequenceClassification, TrainingArguments, Trainer

# Load the GPT-2 model
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=len(df['cluster'].unique()))
model.config.pad_token_id = model.config.eos_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-20-b5cc9093c0c4>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,No log,1.382504
2,No log,0.832291
3,No log,0.712076


<ipython-input-20-b5cc9093c0c4>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=150, training_loss=1.0368102010091145, metrics={'train_runtime': 3490.4904, 'train_samples_per_second': 0.688, 'train_steps_per_second': 0.043, 'total_flos': 156779465932800.0, 'train_loss': 1.0368102010091145, 'epoch': 3.0})

In [22]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

# Generate predictions
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Decode predictions and true labels
predicted_texts = [tokenizer.decode(tokenizer.encode(str(label))) for label in predicted_labels]
true_texts = [tokenizer.decode(tokenizer.encode(str(label))) for label in test_df['cluster']]

# Calculate BLEU score
bleu_scores = []
for pred, true in tqdm(zip(predicted_texts, true_texts), total=len(predicted_texts)):
    bleu_scores.append(sentence_bleu([true.split()], pred.split()))

average_bleu = np.mean(bleu_scores)
print(f"Average BLEU score: {average_bleu}")

<ipython-input-20-b5cc9093c0c4>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or us

Average BLEU score: 1.3754831520312335e-231


In [ ]:
from huggingface_hub import notebook_login

notebook_login()